In [1]:
import numpy as np
import pandas as pd
import env
import matplotlib.pyplot as plt
import seaborn as sns


In [7]:
query = """
    SELECT p.id, transactiondate, parcelid, bathroomcnt, bedroomcnt, calculatedfinishedsquarefeet as squarefeet,
    fips, latitude,  longitude, regionidcity as city, regionidcounty as county, regionidzip as zipcode, yearbuilt, taxvaluedollarcnt as price FROM predictions_2017 p
JOIN properties_2017 USING (parcelid)
WHERE transactiondate BETWEEN '2017-05-01' AND '2017-06-30'
AND bedroomcnt >0 AND bathroomcnt>0 AND `propertylandusetypeid` = 261
    """

In [8]:
df = pd.read_sql(query, env.get_db_url('zillow'))

In [9]:
df.head()

,id,transactiondate,parcelid,bathroomcnt,bedroomcnt,squarefeet,fips,latitude,longitude,city,county,zipcode,yearbuilt,price
0,1248,2017-06-23,11289917,2.0,3.0,1458.0,6037.0,34686163.0,-118113100.0,5534.0,3101.0,97318.0,1970.0,136104.0
1,1772,2017-06-30,11705026,1.0,2.0,1421.0,6037.0,33999877.0,-118291863.0,12447.0,3101.0,96018.0,1911.0,35606.0
2,2028,2017-06-01,14269464,3.0,4.0,2541.0,6059.0,33694636.0,-117912245.0,38032.0,1286.0,96958.0,2003.0,880456.0
3,3273,2017-06-01,11389003,2.0,3.0,1650.0,6037.0,33985377.0,-118361620.0,396550.0,3101.0,96037.0,1949.0,614000.0
4,3429,2017-06-29,11967869,1.0,2.0,693.0,6037.0,34112946.0,-118208106.0,12447.0,3101.0,96023.0,1921.0,274237.0
